In [1]:
import pandas as pd

In [ ]:
## let's examine the provided dataframes for the reference genomes.
reference_taxonomy_df = pd.read_pickle("../data/SGVFinder/DataFiles/representatives.genomes.taxonomy.df")
## The annotations are in a multiindexed dataframe:
## https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.html
## https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#hierarchical-indexing-multiindex
reference_annotations_df = pd.read_pickle("../data/SGVFinder/DataFiles/representatives.genes.drepped.annotations.df")

In [ ]:
reference_taxonomy_df

In [ ]:
reference_annotations_df

In [ ]:
## rows are samples, and the columns are the reference genomes.
## I am not yet sure how to get the genes or regions.

orig_deletion_df = pd.read_pickle("../results/DIABIMMUNE-SGVFinder/orig_dsgv.df")
orig_struct_variation_df = pd.read_pickle("../results/DIABIMMUNE-SGVFinder/orig_vsgv.df")

DIABIMMUNE_deletion_df = pd.read_pickle("../results/DIABIMMUNE-SGVFinder/dsgv.df")
DIABIMMUNE_struct_variation_df = pd.read_pickle("../results/DIABIMMUNE-SGVFinder/vsgv.df")

In [ ]:
## let's get a list of all the reference genomes present in my DIABIMMUNE analysis,
## based on the column names.
DIABIMMUNE_deletion_cols = [str(col) for col in DIABIMMUNE_deletion_df.columns]
DIABIMMUNE_deletion_reference_genomes = set([x.split(':')[0] for x in DIABIMMUNE_deletion_cols])

DIABIMMUNE_structvar_cols = [str(col) for col in DIABIMMUNE_struct_variation_df.columns]
DIABIMMUNE_structvar_reference_genomes = set([x.split(':')[0] for x in DIABIMMUNE_structvar_cols])

## now take the union of the two sets. There are 38 reference genomes represented here.
DIABIMMUNE_ref_genomes = DIABIMMUNE_structvar_reference_genomes.union(DIABIMMUNE_deletion_reference_genomes)
num_DIABIMMUNE_ref_genomes = len(DIABIMMUNE_ref_genomes)

## now take the intersection of the two sets. There are 38 reference genomes represented here.
DIABIMMUNE_ref_intersection = DIABIMMUNE_structvar_reference_genomes.intersection(DIABIMMUNE_deletion_reference_genomes)
num_DIABIMMUNE_ref_intersection = len(DIABIMMUNE_ref_intersection)

## the union and intersection are the same!
print(DIABIMMUNE_ref_intersection == DIABIMMUNE_ref_genomes)

In [ ]:
## E. coli is 562.PRJNA242851.
Ecoli_reference = "562.PRJNA242851"

## let's get all columns that match E. coli in the deletion dataframe.
deletion_cols_with_Ecoli = [x for x in DIABIMMUNE_deletion_cols if Ecoli_reference in x]

## let's get all columns that match E. coli in the structural variation dataframe.
structvar_cols_with_Ecoli = [x for x in DIABIMMUNE_structvar_cols if Ecoli_reference in x]

## now let's select just these Ecoli columns in both dataframes,
## and drop rows that only have missing values
Ecoli_DIABIMMUNE_deletion_df = DIABIMMUNE_deletion_df[deletion_cols_with_Ecoli].dropna(how="all")
Ecoli_DIABIMMUNE_structvar_df = DIABIMMUNE_struct_variation_df[structvar_cols_with_Ecoli].dropna(how="all")

In [ ]:
Ecoli_DIABIMMUNE_deletion_df

In [ ]:
Ecoli_DIABIMMUNE_structvar_df

In [ ]:
## Let's subset the original data by E. coli for comparison.
orig_sv_cols = [str(x) for x in orig_struct_variation_df.columns]

## let's get all columns that match E. coli in the structural variation dataframe.
orig_sv_cols_with_Ecoli = [x for x in orig_sv_cols if Ecoli_reference in x]

## now let's select just these Ecoli columns in both dataframes.
Ecoli_orig_sv_df = orig_struct_variation_df[orig_sv_cols_with_Ecoli].dropna(how="all")
Ecoli_orig_sv_df

In [ ]:
## Let's examine the E. coli dataframe from the SGVFinder original data.
## see: https://www.ncbi.nlm.nih.gov/bioproject/PRJNA242851/
## This is E. coli NIH1.

## each number refers to a bin in the reference genome. each bin is 1kbp in length.
Ecoli_df = pd.read_pickle("../data/SGVFinder/DataFiles/orig_frames/562.PRJNA242851.df")
Ecoli_df

In [ ]:
## let's also examine just the Ecoli genes from reference_annotations_df.
Ecoli_annotations_df = reference_annotations_df.loc[Ecoli_reference]
Ecoli_annotations_df

In [ ]:
unknown_protein_keywords = "unknown|Unknown|hypothetical|Hypothetical|Uncharacterized|Uncharacterised|uncharacterized|uncharacterised|DUF|unknow|putative protein in bacteria|Unassigned|unassigned"

## NOTE: some hypothetical proteins are "ISXX family insertion sequence hypothetical protein"
## so filter out those cases, when counting unknown proteins.


## match MGE genes using the following keywords in the "product" annotation
IS_keywords = "IS|transposon|Transposase|transposase|Transposable|transposable|virus|Phage|phage|integrase|Integrase|baseplate|tail|intron|Mobile|mobile|antitoxin|toxin|capsid|plasmid|Plasmid|conjug|Tra"

## Elongation Factor Tu (2 copies in most bacteria).
## \\b is a word boundary.
## see: https://stackoverflow.com/questions/62430498/detecting-whole-words-using-str-detect-in-r
EFTu_keywords = "\\bTu | Tu\\b|-Tu\\b"

## antibiotic-specific keywords.
chloramphenicol_keywords = "chloramphenicol|Chloramphenicol"
tetracycline_keywords = "tetracycline|Tetracycline"
MLS_keywords = "macrolide|lincosamide|streptogramin"
multidrug_keywords = "multidrug"
beta_lactam_keywords = "lactamase"
glycopeptide_keywords = "glycopeptide resistance|VanZ"
polypeptide_keywords = "bacitracin|polymyxin B"
diaminopyrimidine_keywords = "trimethoprim-resistant"
sulfonamide_keywords = "sulfonamide-resistant"
quinolone_keywords = "quinolone|Quinolone|oxacin"
aminoglycoside_keywords = "aminoglycoside|streptomycin|Streptomycin|kanamycin|Kanamycin|tobramycin|Tobramycin|gentamicin|Gentamicin|neomycin|Neomycin"
macrolide_keywords = "macrolide|ketolide|Azithromycin|azithromycin|Clarithromycin|clarithromycin|Erythromycin|erythomycin"

antibiotic_keywords = "chloramphenicol|Chloramphenicol|tetracycline|Tetracycline|macrolide|lincosamide|streptogramin|multidrug|lactamase|glycopeptide resistance|VanZ|bacitracin|polymyxin B|trimethoprim-resistant|sulfonamide-resistant|quinolone|Quinolone|oxacin|aminoglycoside|streptomycin|Streptomycin|kanamycin|Kanamycin|tobramycin|Tobramycin|gentamicin|Gentamicin|neomycin|Neomycin|macrolide|ketolide|Azithromycin|azithromycin|Clarithromycin|clarithromycin|Erythromycin|erythomycin|antibiotic resistance"